In [1]:
import torch as tr

In [2]:
a = tr.tensor([-10, 10, 10])
tr.exp(a) / tr.exp(a).sum()

tensor([1.0306e-09, 5.0000e-01, 5.0000e-01])

sentences = [
  "How are you",
  "Who are you",
  "Who are they",
  "Who are we",
  "Who am I",
  "Who am I",
  "Where are you going"
]

# replace to training dataset from training datas
## 1. all the elements in sentences are integers, elements equal to words in this model

## 2. read all elements from training data with randomized segments having same orders



In [3]:
path1 = "C:/Users/gzkei/PycharmProjects/cis667-secretary-problem/data/first_training_set.csv"
path2 = "C:/Users/gzkei/PycharmProjects/cis667-secretary-problem/data/second_training_set.csv"
path3 = "C:/Users/gzkei/PycharmProjects/cis667-secretary-problem//data/third_training_set.csv"
path4 = "C:/Users/gzkei/PycharmProjects/cis667-secretary-problem//data/fourth_training_set.csv"
path5 = "C:/Users/gzkei/PycharmProjects/cis667-secretary-problem//data/fifth_training_set.csv"


# pick 5000 (each 1000, imply by the parameter inside) segments in each dataset with randomized segments

sentences = []

import number_helpers

sen1 = number_helpers.augment_sentences(path1, 10, 100)
sen2 = number_helpers.augment_sentences(path2, 10, 100)
sen3 = number_helpers.augment_sentences(path3, 10, 100)
sen4 = number_helpers.augment_sentences(path4, 10, 100)
sen5 = number_helpers.augment_sentences(path5, 10, 100)

sentences.extend(sen1)
sentences.extend(sen2)
sentences.extend(sen3)
sentences.extend(sen4)
sentences.extend(sen5)

print(sentences[:5])
print('------------------------------------------------------------------------------------------------------------------------')
print('length:  '+str(len(sentences)))



[['0', '1', '2', '2', '0', '2', '1', '3', '1', '4', '1', '1', '0', '1', '2', '7', '1', '2', '2', '5', '0', '7', '2', '0', '2', '9', '3', '0', '3', '0', '2', '0', '1', '0', '1', '0', '5', '2', '2', '6', '1', '4', '2', '1', '1', '8', '5', '1', '5', '5', '0', '1', '0', '2', '7', '3', '2', '6', '1', '2', '4', '2', '1', '0', '1', '5', '1', '2', '3', '3', '2', '6', '3', '1', '0', '1', '3', '0', '5', '1', '0', '3', '2', '2', '2', '1', '0', '2', '0', '1', '1', '7', '0', '2', '2', '5', '2', '2', '1', '4', '-1'], ['1', '1', '3', '1', '7', '2', '1', '0', '7', '6', '0', '0', '2', '3', '1', '4', '3', '1', '4', '2', '0', '2', '1', '0', '3', '1', '0', '0', '1', '6', '0', '0', '1', '0', '1', '0', '4', '7', '4', '1', '3', '6', '0', '0', '1', '0', '4', '2', '1', '1', '3', '0', '0', '0', '1', '3', '1', '2', '2', '0', '5', '1', '0', '0', '2', '2', '0', '0', '1', '4', '2', '6', '3', '5', '1', '3', '2', '6', '1', '4', '0', '5', '0', '1', '3', '7', '0', '0', '0', '6', '1', '0', '2', '4', '2', '2', '0', '6', 

# start to set building

In [4]:

# Make a dictionary mapping each word to a one-hot tensor
words = set()
for sentence in sentences:
    for word in sentence:
        #for word in sentence.split(" "):
        words.add(word)
words = tuple(words)  # deterministic order

print(words[:5])

# PyTorch LSTM expects 3d tensors representing (sequence length, batch size, number of features)
I = tr.eye(len(words))
dictionary = {
    word: I[w].reshape(1, 1, len(words))
    for w, word in enumerate(words)}

print(len(dictionary))

('3', '6', '8', '7', '9')
11


# try to train the LSTM

In [5]:
# Define a small LSTM recurrent neural network with linear hidden-to-output layer
class Net(tr.nn.Module):
    def __init__(self, hidden_size):
        super(Net, self).__init__()
        self.lstm = tr.nn.LSTM(input_size=len(words), hidden_size=hidden_size)
        self.readout = tr.nn.Linear(in_features=hidden_size, out_features=len(words))

    def forward(self, x, v=None):
        _, v = self.lstm(x) if v is None else self.lstm(x, v)  # update hidden from input
        h, c = v  # LSTM hidden vector and internal so-called "cell state"
        y = self.readout(h)  # get output from hidden
        y = tr.softmax(y, dim=-1)  # make sure output is a probability distribution
        return y, v


print(Net(3))

Net(
  (lstm): LSTM(11, 3)
  (readout): Linear(in_features=3, out_features=11, bias=True)
)


# next

In [ ]:
net = Net(3)
# opt = tr.optim.SGD(net.parameters(), lr=0.001)
opt = tr.optim.Adam(net.parameters(), lr=0.05)

for epoch in range(20000):

    batch_loss = 0.

    for sentence in sentences:
        #tokens = sentence.split(" ")
        tokens = sentence
        v = None  # no hidden activation at first time-step
        for t in range(len(tokens) - 1):
            y, v = net(dictionary[tokens[t]], v)
            y_target = dictionary[tokens[t + 1]]

            loss = tr.sum((y - y_target)**2) # MSE
            #loss = -tr.sum(y_target * tr.log(y))  # Cross-entropy
            batch_loss += loss

    batch_loss.backward()
    opt.step()
    opt.zero_grad()

    print(epoch, batch_loss.item())
    # if epoch % 100 == 0: print(epoch, batch_loss.item())

0 4483.119140625
1 4400.45263671875
2 4332.916015625
3 4278.32080078125
4 4233.96923828125
5 4198.57080078125
6 4172.0830078125
7 4154.97314453125
8 4146.875
9 4145.29443359375
10 4146.05712890625
11 4146.0849609375
12 4144.90625
13 4143.15673828125
14 4141.35009765625
15 4139.90625
16 4139.25
17 4139.43017578125
18 4139.857421875
19 4139.9228515625
20 4139.39404296875
21 4138.50439453125
22 4137.66552734375
23 4137.1396484375
24 4137.01220703125
25 4137.10693359375
26 4137.25830078125
27 4137.2529296875
28 4136.8818359375
29 4136.12353515625
30 4135.083984375
31 4134.0205078125
32 4133.0966796875
33 4132.45947265625
34 4132.06787109375
35 4131.794921875
36 4131.52783203125
37 4131.18701171875
38 4130.76806640625
39 4130.27685546875
40 4129.794921875
41 4129.37255859375
42 4129.05078125
43 4128.89599609375
44 4128.83935546875
45 4128.7880859375
46 4128.66455078125
47 4128.4814453125
48 4128.25927734375
49 4128.0244140625
50 4127.79833984375
51 4127.55712890625
52 4127.3642578125
53 412

406 4057.47509765625
407 4060.232421875
408 4062.754150390625
409 4062.48974609375
410 4057.941650390625
411 4057.76904296875
412 4057.348388671875
413 4057.961669921875
414 4057.4931640625
415 4057.23974609375
416 4056.451171875
417 4056.8974609375
418 4055.87109375
419 4057.327392578125
420 4055.610595703125
421 4055.765380859375
422 4055.666259765625
423 4055.557861328125
424 4055.70458984375
425 4054.863037109375
426 4054.979736328125
427 4054.39794921875
428 4054.416015625
429 4053.897216796875
430 4054.155517578125
431 4054.236083984375
432 4054.2333984375
433 4053.620361328125
434 4055.140380859375
435 4054.88037109375
436 4053.884521484375
437 4054.641845703125
438 4054.04052734375
439 4053.77685546875
440 4053.35791015625
441 4053.420166015625
442 4052.9072265625
443 4053.025634765625
444 4052.95556640625
445 4053.3466796875
446 4054.473388671875
447 4054.70068359375
448 4055.267333984375
449 4053.876220703125
450 4054.31201171875
451 4053.42431640625
452 4054.48291015625
453 

795 4053.55615234375
796 4050.126708984375
797 4056.253173828125
798 4051.027587890625
799 4051.36572265625
800 4048.700927734375
801 4046.0615234375
802 4049.8828125
803 4047.961669921875
804 4050.438720703125
805 4052.0869140625
806 4047.8671875
807 4052.08740234375
808 4048.675048828125
809 4048.160888671875
810 4046.215087890625
811 4047.3076171875
812 4047.873046875
813 4049.247314453125
814 4069.072265625
815 4082.227294921875
816 4078.53955078125
817 4063.5654296875
818 4075.7451171875
819 4086.58935546875
820 4087.84423828125
821 4084.355224609375
822 4081.6650390625
823 4078.76171875
824 4082.708984375
825 4080.490966796875
826 4074.15478515625
827 4072.34814453125
828 4070.589599609375
829 4067.187255859375
830 4067.864990234375
831 4066.580810546875
832 4065.508056640625
833 4063.391845703125
834 4061.307861328125
835 4060.189697265625
836 4060.249267578125
837 4058.156494140625
838 4056.853515625
839 4056.245849609375
840 4053.432861328125
841 4054.668212890625
842 4052.473

1186 4111.4091796875
1187 4111.4462890625
1188 4111.3466796875
1189 4111.26025390625
1190 4111.20751953125
1191 4111.203125
1192 4111.18212890625
1193 4111.11376953125
1194 4111.03466796875
1195 4110.9638671875
1196 4110.92919921875
1197 4110.865234375
1198 4110.830078125
1199 4110.74267578125
1200 4110.68212890625
1201 4110.64013671875
1202 4110.55517578125
1203 4110.48974609375
1204 4110.4306640625
1205 4110.35302734375
1206 4110.2314453125
1207 4110.40283203125
1208 4110.32568359375
1209 4111.11572265625
1210 4111.4638671875
1211 4111.5283203125
1212 4111.515625
1213 4111.32861328125
1214 4110.99072265625
1215 4110.70263671875
1216 4110.43212890625
1217 4109.99462890625
1218 4110.2421875
1219 4110.08740234375
1220 4109.8359375
1221 4109.75
1222 4109.623046875
1223 4109.66796875
1224 4109.7705078125
1225 4109.42529296875
1226 4109.1875
1227 4108.9755859375
1228 4108.95166015625
1229 4109.21240234375
1230 4109.07421875
1231 4109.26513671875
1232 4109.021484375
1233 4108.587890625
1234

1575 4091.989990234375
1576 4091.935302734375
1577 4091.8671875
1578 4091.806884765625
1579 4091.744140625
1580 4091.69482421875
1581 4091.649658203125
1582 4091.605224609375
1583 4091.54541015625
1584 4091.513671875
1585 4091.4521484375
1586 4091.435302734375
1587 4091.393310546875
1588 4091.3564453125
1589 4091.32080078125
1590 4091.287841796875
1591 4091.2548828125
1592 4091.21875
1593 4091.197509765625
1594 4091.155517578125
1595 4091.132568359375
1596 4091.103271484375
1597 4091.068603515625
1598 4091.03125
1599 4090.999755859375
1600 4090.9755859375
1601 4090.953369140625
1602 4090.917724609375
1603 4090.886962890625
1604 4090.8544921875
1605 4090.817138671875
1606 4090.7880859375
1607 4090.75537109375
1608 4090.71923828125
1609 4090.689453125
1610 4090.654296875
1611 4090.625
1612 4090.595947265625
1613 4090.5576171875
1614 4090.51953125
1615 4090.483642578125
1616 4090.443115234375
1617 4090.4052734375
1618 4090.365478515625
1619 4090.34033203125
1620 4090.304443359375
1621 409

...

3482 4119.89892578125
3483 4119.89306640625
3484 4119.884765625
3485 4119.8681640625
3486 4119.8505859375
3487 4119.830078125
3488 4119.82958984375
3489 4119.8251953125
3490 4119.82666015625
3491 4119.8125
3492 4119.8037109375
3493 4119.79736328125
3494 4119.78369140625
3495 4119.78076171875
3496 4119.7587890625
3497 4119.75732421875
3498 4119.74072265625
3499 4119.72509765625
3500 4119.71435546875
3501 4119.7197265625
3502 4119.71435546875
3503 4119.7021484375
3504 4119.697265625
3505 4119.68798828125
3506 4119.69287109375
3507 4119.69140625
3508 4119.68896484375
3509 4119.6689453125
3510 4119.66015625
3511 4119.6552734375
3512 4119.67626953125
3513 4119.6611328125
3514 4119.64111328125
3515 4119.64453125
3516 4119.64013671875
3517 4119.62890625
3518 4119.63623046875
3519 4119.61865234375
3520 4119.6201171875
3521 4119.60888671875
3522 4119.6201171875
3523 4119.61083984375
3524 4119.59619140625
3525 4119.59033203125
3526 4119.591796875
3527 4119.5849609375
3528 4119.58203125
3529 4119

3873 4117.56103515625
3874 4117.52734375
3875 4117.537109375
3876 4117.548828125
3877 4117.51904296875
3878 4117.51708984375
3879 4117.544921875
3880 4117.501953125
3881 4117.50927734375
3882 4117.52685546875
3883 4117.49560546875
3884 4117.50244140625
3885 4117.50439453125
3886 4117.48486328125
3887 4117.4951171875
3888 4117.47802734375
3889 4117.474609375
3890 4117.478515625
3891 4117.45556640625
3892 4117.458984375
3893 4117.46875
3894 4117.44384765625
3895 4117.4404296875
3896 4117.43896484375
3897 4117.4169921875
3898 4117.419921875
3899 4117.41064453125
3900 4117.39990234375
3901 4117.40087890625
3902 4117.39208984375
3903 4117.3857421875
3904 4117.3818359375
3905 4117.37451171875
3906 4117.36962890625
3907 4117.36181640625
3908 4117.36083984375
3909 4117.36962890625
3910 4117.353515625
3911 4117.36279296875
3912 4117.33935546875
3913 4117.35693359375
3914 4117.32373046875
3915 4117.34033203125
3916 4117.341796875
3917 4117.30615234375
3918 4117.3046875
3919 4117.30029296875
3920

4264 4112.501953125
4265 4112.4931640625
4266 4112.4814453125
4267 4112.4716796875
4268 4112.4560546875
4269 4112.43896484375
4270 4112.44140625
4271 4112.4169921875
4272 4112.4150390625
4273 4112.38916015625
4274 4112.3779296875
4275 4112.36962890625
4276 4112.3447265625
4277 4112.318359375
4278 4112.3046875
4279 4112.29150390625
4280 4112.28173828125
4281 4112.2744140625
4282 4112.23779296875
4283 4112.2236328125
4284 4112.205078125
4285 4112.19384765625
4286 4112.1796875
4287 4112.16943359375
4288 4112.15380859375
4289 4112.13671875
4290 4112.1298828125
4291 4112.10009765625
4292 4112.076171875
4293 4112.0615234375
4294 4112.0576171875
4295 4112.03515625
4296 4112.02001953125
4297 4112.00244140625
4298 4111.97900390625
4299 4111.96875
4300 4111.9443359375
4301 4111.93408203125
4302 4111.90185546875
4303 4111.88232421875
4304 4111.8583984375
4305 4111.84033203125
4306 4111.81396484375
4307 4111.78369140625
4308 4111.73291015625
4309 4111.69189453125
4310 4111.55029296875
4311 4111.26

4655 4105.98046875
4656 4105.97119140625
4657 4105.966796875
4658 4105.96435546875
4659 4105.93115234375
4660 4105.927734375
4661 4105.90771484375
4662 4105.9111328125
4663 4105.9052734375
4664 4105.90478515625
4665 4105.87109375
4666 4105.875
4667 4105.84375
4668 4105.83837890625
4669 4105.83935546875
4670 4105.83837890625
4671 4105.8173828125
4672 4105.80615234375
4673 4105.77099609375
4674 4105.78076171875
4675 4105.7724609375
4676 4105.74755859375
4677 4105.74609375
4678 4105.72412109375
4679 4105.724609375
4680 4105.71875
4681 4105.703125
4682 4105.68896484375
4683 4105.6748046875
4684 4105.66064453125
4685 4105.6708984375
4686 4105.6572265625
4687 4105.62451171875
4688 4105.63232421875
4689 4105.6103515625
4690 4105.59716796875
4691 4105.58349609375
4692 4105.591796875
4693 4105.587890625
4694 4105.57373046875
4695 4105.5498046875
4696 4105.54541015625
4697 4105.53125
4698 4105.52587890625
4699 4105.5146484375
4700 4105.5078125
4701 4105.48876953125
4702 4105.4775390625
4703 4105

5048 4101.00634765625
5049 4100.99365234375
5050 4100.970703125
5051 4100.92626953125
5052 4100.87060546875
5053 4100.80859375
5054 4100.7353515625
5055 4100.70166015625
5056 4100.681640625
5057 4100.61962890625
5058 4100.58056640625
5059 4100.5400390625
5060 4100.4755859375
5061 4100.47705078125
5062 4100.40966796875
5063 4100.42333984375
5064 4100.3310546875
5065 4100.3115234375
5066 4100.26904296875
5067 4100.2509765625
5068 4100.24658203125
5069 4100.19970703125
5070 4100.17919921875
5071 4100.14013671875
5072 4100.099609375
5073 4099.9716796875
5074 4099.77685546875
5075 4099.62744140625
5076 4099.44287109375
5077 4099.6240234375
5078 4101.3408203125
5079 4101.72314453125
5080 4101.638671875
5081 4102.88037109375
5082 4103.11328125
5083 4102.146484375
5084 4102.47265625
5085 4101.70556640625
5086 4101.3642578125
5087 4100.97998046875
5088 4101.24658203125
5089 4100.07568359375
5090 4099.85205078125
5091 4100.03173828125
5092 4101.7607421875
5093 4100.84619140625
5094 4100.97753906

# complete epoch
### and try prediction

In [8]:
# Try predicting
word = "3"
v = None
print(word)

for t in range(3):
  x = dictionary[word]
  y, v = net(dictionary[tokens[t]], v)
  y = y.squeeze() # ignore singleton dimensions for time-step/example
  w = y.argmax()
  word = words[w]
  prob = y[w]
  print(word, prob.item())

3
0 0.23600423336029053
1 0.23719550669193268
0 0.23819178342819214
